# Bar data

In [7]:
import numpy as np
import pandas as pd
import datetime as dt
import os
from Scripts.Utils import trading_utils

In [8]:




def UpdateRealizedData(symbol, data_path, symbol_kind, interval_time_list, realized_days_list, implied_theo_ratios_dict):
    realized_dict = {}
    earnings_dates_list = GetEarningsDatesList(symbol, data_path, symbol_kind)
    for interval_time in interval_time_list:
        interval_time_string = interval_time.replace(" ", "_")
        file_path = data_path + "/" + symbol_kind + "/" + symbol + "/" + interval_time_string + ".csv"
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['date'])
        df['date'] = pd.to_datetime(df['datetime']).dt.date
        df['change'] = df.close - df.close.shift(1)
        df['prev_datetime'] = df['datetime'].shift(1)
        df['prev_date'] = pd.to_datetime(df['prev_datetime']).dt.date
        df = df[1:]
        df['tte_from_prev_entry'] = df.apply(lambda x: trading_utils.tte(x['prev_datetime'], x['datetime']), axis =1)
        df['percent_change'] = abs(df.change/df.open)
        df['variance'] = df['percent_change'] * df['percent_change']
        df['time_in_years_till_now'] = df.apply(lambda x : trading_utils.tte(x['datetime'], df['datetime'].iloc[-1]), axis = 1)
        df['time_in_years_till_now'] = df['time_in_years_till_now'].shift(1)
        df['time_in_days_till_now'] = df['time_in_years_till_now'] * 252.0
        df = df[1:]
        df = df.reset_index()


        for num_day in realized_days_list:
            idx = df['time_in_days_till_now'].sub(num_day).abs().idxmin()
            subset_df = df[idx:]
            dict_key = str(num_day)+'_day_interval_'+ interval_time_string
            total_variance = subset_df.variance.sum()
            total_time = subset_df.tte_from_prev_entry.sum()
            annualized_variance = total_variance / total_time
            annualized_vol= annualized_variance **0.5
            realized_dict[dict_key] = annualized_vol

            subset_df = subset_df[~subset_df['date'].isin(earnings_dates_list)]
            subset_df = subset_df[~subset_df['prev_date'].isin(earnings_dates_list)]
            dict_key = str(num_day)+'_day_interval_without_earnings_'+ interval_time_string
            total_variance = subset_df.variance.sum()
            total_time = subset_df.tte_from_prev_entry.sum()
            annualized_variance = total_variance / total_time
            annualized_vol= annualized_variance **0.5
            realized_dict[dict_key] = annualized_vol

        for implied_theo_day in implied_theo_ratios_dict:
            dict_key = str(implied_theo_day) + '_day_implied_' + interval_time_string
            realized_dict[dict_key] = CalculateImpliedTheo(implied_theo_ratios_dict[implied_theo_day], realized_dict, interval_time_string)




    realized_dict['timestamp'] = dt.datetime.now()

    AddToRealizedFile(realized_dict, symbol, data_path, symbol_kind)
    return realized_dict


def CalculateImpliedTheo(implied_theo_ratios_dict, realized_dict, interval_time_string):
    implied_theo = 0
    for num_day in implied_theo_ratios_dict:
        weight = implied_theo_ratios_dict[num_day]
        realized_dict_key = str(num_day)+'_day_interval_without_earnings_'+ interval_time_string
        realized_for_num_day = realized_dict[realized_dict_key]
        implied_theo = implied_theo + weight * realized_for_num_day
        # print("weight for " + str(num_day)  + " = " +str(weight) + ". Realized for " + str(num_day) + " = "  + str(realized_for_num_day))
    return implied_theo


def AddToRealizedFile(realized_dict, symbol, data_path, symbol_kind):
    realized_df = pd.DataFrame(realized_dict, index=[0])
    file_path = data_path + "/" + symbol_kind + "/" + symbol + "/realized.csv"
    file_exist = os.path.exists(file_path)
    if file_exist:
        realized_df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        realized_df.to_csv(file_path, index = False)


def GetEarningsDatesList(symbol, data_path, symbol_kind):
    file_path = data_path + "/" + symbol_kind + "/" + symbol + "/earnings.csv"
    file_df = pd.read_csv(file_path)
    datetime_converted = pd.to_datetime(file_df.date).dt.date.to_list()
    return datetime_converted

def GetInputs(inputs_path):
    df = pd.read_csv(inputs_path, header= None)
    df_dict = dict(zip(df[0], df[1]))
    df_dict['symbols'] = df_dict['symbols'].split("|")
    return df_dict

def GetImpliedRatios(implied_ratios_path):
    df = pd.read_csv(implied_ratios_path)
    realized_days_list = df.historical.to_list()
    df = df.set_index('historical')
    return (df.to_dict(), realized_days_list)




In [9]:
inputs_path = "./../../Inputs/historical_data_inputs.csv"
implied_ratios_path = "./../../Inputs/implied_theo_ratios.csv"
data_path = "./../../HistoricalData"
interval_time_list = ['1 day', '1 hour', '10 mins']
inputs_dict = GetInputs(inputs_path)
(implied_theo_ratios_dict, realized_days_list) = GetImpliedRatios(implied_ratios_path)
for symbol in inputs_dict['symbols']:
    UpdateRealizedData(symbol, data_path, "stocks", interval_time_list, realized_days_list, implied_theo_ratios_dict)
